In [1]:
import pandas as pd
import os
from hydra import initialize, compose
from omegaconf import OmegaConf

from deep_model import UTSDataset
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler

from torch.utils.data import DataLoader
import torch

from utils import normalize_data, separate_data
from quantum_model import QuantumCircuit
from deep_model import MeanSquareError, Net, train_model_deep

In [2]:
HOME_PATH = "../"
with initialize(config_path="../configs/"):
    data_cfg = compose(config_name="data_path")
data_cfg = OmegaConf.create(data_cfg)

/tmp/ipykernel_43530/4189321069.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../configs/"):


In [3]:
filter_uts_training_path = os.path.join(HOME_PATH, data_cfg.UTS_data.processed_data.filter_train)
filter_uts_testing_path = os.path.join(HOME_PATH, data_cfg.UTS_data.processed_data.filter_test)

save_model_path = os.path.join(HOME_PATH, data_cfg.model_train.best_model_state)

In [4]:
data_train = pd.read_csv(filter_uts_training_path).drop(columns=["Time"])
data_test = pd.read_csv(filter_uts_testing_path).drop(columns=["Time"])

In [5]:
# normalize data from dataframe
uts_filter_train_data = normalize_data(data_train)
uts_filter_test_data = normalize_data(data_test)

# separate data from dataframe
X_train, y_train = separate_data(uts_filter_train_data)
X_test, y_test = separate_data(uts_filter_test_data)

# standard data
scl = StandardScaler()
X_train_scaled = scl.fit_transform(X_train)
X_test_scaled = scl.transform(X_test)

# transformer PCA
transformer = KernelPCA(n_components=64, kernel="cosine", random_state=100)
# transformer = PCA(n_components=0.91, random_state=100)
X_train_transformed = transformer.fit_transform(X_train_scaled)
X_test_transformed = transformer.transform(X_test_scaled)

X_train_transformed.shape, X_test_transformed.shape

((9108, 64), (388, 64))

In [ ]:
datasets = {
    "train": UTSDataset(X_train_transformed, y_train),
    "val": UTSDataset(X_test_transformed, y_test)
}

dataloaders = {
    "train": DataLoader(datasets["train"], batch_size=16, shuffle=True, num_workers=os.cpu_count()),
    "val": DataLoader(datasets["val"], batch_size=1, shuffle=True, num_workers=os.cpu_count())
}

dataset_sizes = {
    "train": len(datasets["train"]), 
    "val": len(datasets["val"])
}

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(dataloaders["train"]))
test_features, test_labels = next(iter(dataloaders["val"]))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [ ]:
import qiskit
import torch
simulator = qiskit.Aer.get_backend('aer_simulator')
circuit = QuantumCircuit(1, simulator, 512, is_add_noise=True)
print('Expected value for rotation pi {}'.format(circuit.forward(torch.rand(9))))
circuit.circuit.draw('mpl')
# circuit.plot(torch.rand(9))

In [ ]:
model = Net(quantum=True)
# model.load_state_dict(torch.load(save_model_path))

# optimizer = torch.optim.SGD(model.parameters(), lr=0.004, momentum=0.9, nesterov=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.004, weight_decay=0.001)

decayRate = 0.96
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

loss_func = MeanSquareError()

In [ ]:
train_model_deep(model, loss_func, optimizer, dataloaders, dataset_sizes, save_model_path, num_epochs=50)